In [14]:
import os
import base64
from llm_code.prompt_generation import PromptGenerator
from llm_code.nutrition_api_calls import NutritionExtraction 
from functions.nutrition_processing import *
from functions.nutrition_image import *
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### set up queries


In [15]:
weight = 100
name = 'mexican inspired chilli ready meal'
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# Path to your image
image_path = "/Users/jasperhajonides/Documents/Projects/website/dash_health_app/images/IMG_6916 2.HEIC"
image_input = encode_image(image_path)
base64_img, file_format = resize_and_crop_image(image_input)


### Prompts


In [16]:
# generate prompts
pg = PromptGenerator()
prompts = pg.generate_prompts(name, weight,)


### text

In [122]:
# prepare api for call
nutrition_dict, missing_keys = nutrition.openai_api(prompt = prompts['missing_details_text_prompt'])

 self.full_nutr_dict!!!!!!!!! {'macros': ['protein', 'fat', 'carbohydrates'], 'macros_detailed': ['fiber', 'saturated fat', 'unsaturated fat', 'sugar', 'essential amino acids', 'nonessential amino acids'], 'sugar_types': ['glucose', 'fructose', 'galactose', 'lactose'], 'fiber_types': ['soluble fiber', 'insoluble fiber'], 'fat_types': ['cholesterol'], 'minerals': ['iron', 'magnesium', 'zinc', 'calcium', 'potassium', 'sodium'], 'minerals_detailed': ['phosphorus', 'copper', 'manganese', 'selenium', 'chromium', 'molybdenum', 'iodine'], 'essential amino acids': ['histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine'], 'conditionally essential amino acids': ['arginine', 'cysteine', 'glutamine', 'glycine', 'proline', 'tyrosine'], 'nonessential amino acids': ['alanine', 'aspartic acid', 'asparagine', 'glutamic acid', 'serine', 'selenocysteine', 'pyrrolysine'], 'vitamins': ['vitamin a', 'vitamin c', 'vitamin d', 'vitamin e', 'vitamin k

In [17]:
# prepare api for call
nutrition = NutritionExtraction({})
nutrition_dict, missing_keys = nutrition.openai_api(prompt = prompts['text_prompt'])

 self.full_nutr_dict!!!!!!!!! {'indices': ['glycemic index', 'calories', 'weight'], 'macros': ['protein', 'fat', 'carbohydrates'], 'macros_detailed': ['fiber', 'saturated fat', 'unsaturated fat', 'sugar'], 'sugar_types': ['glucose', 'fructose', 'galactose', 'lactose'], 'fiber_types': ['soluble fiber', 'insoluble fiber'], 'fat_types': ['cholesterol'], 'essential amino acids': ['histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine'], 'conditionally essential amino acids': ['arginine', 'cysteine', 'glutamine', 'glycine', 'proline', 'tyrosine'], 'nonessential amino acids': ['alanine', 'aspartic acid', 'asparagine', 'glutamic acid', 'serine', 'selenocysteine', 'pyrrolysine'], 'minerals': ['iron', 'magnesium', 'zinc', 'calcium', 'potassium', 'sodium'], 'minerals_detailed': ['phosphorus', 'copper', 'manganese', 'selenium', 'chromium', 'molybdenum', 'iodine'], 'vitamins': ['vitamin a', 'vitamin c', 'vitamin d', 'vitamin e', 'vitamin k

In [26]:
nutrition_dict

{'name': 'Mexican Inspired Chilli Ready Meal',
 'weight': 100.0,
 'description': 'A spicy and flavorful combination of beans, ground beef, tomatoes, and peppers',
 'calories': 200.0,
 'carbohydrates': 25.0,
 'protein': 15.0,
 'fat': 8.0,
 'saturated fat': 3.0,
 'unsaturated fat': 5.0,
 'sugar': 5.0,
 'glycemic index': 50.0,
 'fiber': 8.0,
 'iron': 2.0}

In [120]:
nutrition_dict, missing_keys

({'name': 'Mexican Inspired Chilli Ready Meal',
  'weight': '100g',
  'description': 'A ready-to-eat meal with a blend of Mexican spices, beans, and meat',
  'calories': 250.0,
  'carbohydrates': 25.0,
  'protein': 15.0,
  'fat': 10.0,
  'saturated fat': 3.5,
  'unsaturated fat': 6.5,
  'sugar': 5.0,
  'glucose': 2.0,
  'fructose': 1.0,
  'lactose': 0.0,
  'galactose': 0.0,
  'glycemic index': 55.0,
  'fiber': 8.0,
  'soluble fiber': 2.0,
  'insoluble fiber': 6.0,
  'iron': 2.0,
  'histidine': 0.5,
  'isoleucine': 1.0,
  'leucine': 2.0,
  'lysine': 1.5,
  'methionine': 0.8,
  'phenylalanine': 1.2,
  'threonine': 1.0,
  'tryptophan': 0.3,
  'valine': 1.5,
  'essential amino acids': 9.8},
 ['nonessential amino acids'])

In [28]:
def adjust_child_values_to_not_exceed_parent(json_entry):
    # Define parent-child relationships
    parent_child_map = {
            'carbohydrates': ['fiber', 'sugar'],
            'protein': ['nonessential amino acids', 'essential amino acids'],
            'sugar': ['glucose', 'fructose', 'galactose', 'lactose'],
            'fiber': ['soluble fiber', 'insoluble fiber'],
            'fat': ['saturated fat', 'unsaturated fat', 'cholesterol'],
            'essential amino acids': ['histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine'],
            'conditionally essential amino acids': ['arginine', 'cysteine', 'glutamine', 'glycine', 'proline', 'tyrosine'],
            'nonessential amino acids': ['alanine', 'aspartic acid', 'asparagine', 'glutamic acid', 'serine', 'selenocysteine', 'pyrrolysine']
    
    }

    # Adjust child values if their sum exceeds the parent's value
    for parent, children in parent_child_map.items():
        parent_value = json_entry.get(parent, 0)
        child_values = {child: json_entry.get(child, 0) for child in children}
        total_child_value = sum(child_values.values())

        # If the total of child values exceeds the parent value, scale them down
        if total_child_value > parent_value and parent_value > 0:
            scale_factor = parent_value / total_child_value
            print('rescaling :', scale_factor)
            for child, value in child_values.items():
                # Scale down each child's value
                json_entry[child] = value * scale_factor

    return json_entry


json_entry = adjust_child_values_to_not_exceed_parent(nutrition_dict)

### image

In [155]:



input_desc = 'vegetarian mexican inspired curry with sweet potato'


# weight = 100
# name = 'small biscuit'

# run image extraction 
nutrition = NutritionExtraction(detail='core')

# generate prompts
pg = PromptGenerator(nutritionextractor=nutrition)
prompts = pg.generate_prompts(name, weight)

# run api
if input_desc is not None:
    prompts['image_text_prompt'] = prompts['image_text_prompt'] + f' The item in the picture is a {input_desc}'
nutrition_dict, missing_keys = nutrition.openai_api_image(prompt = prompts['image_text_prompt'], image=base64_img, n=2)

found nutrition class
WE ARE GOING TO GENERATE THESE VALS ['glycemic index', 'calories', 'protein', 'fat', 'carbohydrates', 'fiber', 'saturated fat', 'unsaturated fat', 'sugar', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine']


In [156]:
nutrition_dict

{'llm_output': 'To estimate the serving size and nutritional values, I\'ll have to make an educated guess based on what I can observe and assume about the ingredients and their typical portion sizes. Please remember that these numbers could vary significantly in reality.\n\n```json\n{\n  "name item": "Vegetarian Mexican Inspired Curry with Sweet Potato",\n  "description": "A curry dish featuring sweet potatoes, black beans, corn, and rice in a savory and possibly spiced sauce.",\n  "weight": 350,\n  "calories": 550,\n  "carbohydrates": 100,\n  "sugar": 10,\n  "fiber": 10,\n  "protein": 15,\n  "fat": 10,\n  "saturated fat": 3,\n  "unsaturated fat": 6,\n  "cholesterol": 0,\n  "glycemic index": 65\n}\n```\n\nKeep in mind that these values are approximations, especially without more specific information on the precise ingredients and amounts used in the dish. The weight is estimated as a typical serving size for a main dish, and the calories and other macronutrients are judged based on the

In [173]:



input_desc = 'vegetarian mexican inspired curry with sweet potato'


# weight = 100
# name = 'small biscuit'

# run image extraction 
nutrition = NutritionExtraction(detail='core')

# generate prompts
pg = PromptGenerator(nutritionextractor=nutrition)
prompts = pg.generate_prompts(name, weight)

# run api
if input_desc is not None:
    prompts['image_text_prompt'] = prompts['image_text_prompt'] + f' The item in the picture is a {input_desc}'
nutrition_dict, missing_keys = nutrition.openai_api_image(prompt = prompts['image_text_prompt'], image=base64_img, n=1)

found nutrition class
WE ARE GOING TO GENERATE THESE VALS ['glycemic index', 'calories', 'weight', 'protein', 'fat', 'carbohydrates', 'fiber', 'saturated fat', 'unsaturated fat', 'sugar', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine']


In [176]:
nutrition_dict

{'llm_output': 'To estimate the serving size and nutritional values of the vegetarian Mexican-inspired curry with sweet potato in the picture, we will make some assumptions based on typical serving sizes and ingredients. Please keep in mind that actual nutritional values can vary greatly depending on the specific ingredients and quantities used in the dish.\n\nLet\'s assume this is a standard serving size of approximately 1 cup (about 235 ml) of curry paired with 1 cup of cooked rice (roughly 195 grams). Now, we\'ll estimate the nutritional content based on this serving size.\n\n```json\n{\n  "name": "Vegetarian Mexican-Inspired Curry with Sweet Potato",\n  "description": "A dish consisting of a spiced curry sauce with sweet potatoes, likely including ingredients such as beans, corn, peppers, and other vegetables served with rice.",\n  "glycemic index": 50,\n  "calories": 350,\n  "weight": 430,\n  "protein": 8,\n  "fat": 10,\n  "carbohydrates": 60,\n  "fiber": 8,\n  "saturated fat": 3,

prompts['image_text_prompt'] 

In [15]:

# generate prompts
pg = PromptGenerator(nutritionextractor=nutrition)
prompts = pg.generate_prompts(name, weight)

# run api
nutrition_dict, missing_keys = nutrition.openai_api_image(prompt = prompts['missing_details_image_prompt'], image=base64_img, n=1)

NameError: name 'nutrition' is not defined

In [165]:
prompts['missing_details_image_prompt']

"  We're going to play a game, we're going to estimate some nutritional contents of 425.0g of None.\n            lets estimate the content of ['histidine', 'isoleucine', 'leucine', 'lysine', 'methionine', 'phenylalanine', 'threonine', 'tryptophan', 'valine'] in grams. return in json format.\n            "

In [164]:
nutrition_dict

{'llm_output': 'To estimate the nutritional values, we\'ll start by identifying the foods shown in the picture and then we\'ll use standard nutritional information for those ingredients to estimate the values for the serving size depicted.\n\nBased on the image, we see a vegetarian Mexican-inspired curry with sweet potato, which appears to include sweet potatoes, black beans, corn, and rice, enveloped in a tomato-based sauce, possibly with various spices and herbs that are typical for curry dishes.\n\nLet\'s estimate the serving size and nutritional values. Please note that these are rough estimates:\n\n- Sweet potatoes: 150g\n- Black beans: 50g\n- Corn: 50g\n- Rice: 150g\n- Tomato-based curry sauce: 100g\n\nThe following nutritional values are estimated based on the assumed quantities and typical values for each component:\n\n```json\n{\n  "name item": "Vegetarian Mexican Inspired Curry with Sweet Potato",\n  "description": "A plate of vegetarian curry featuring sweet potatoes, black 

In [33]:
processed_data

{'llm_output': '```json\n{\n  "name item": "Marie biscuit",\n  "description of food content": "A simple, round, thin wheat flour-based biscuit with sugar and vanilla flavor",\n  "grams in picture": "7",\n  "total calories": "30",\n  "carbohydrates": "4.5",\n  "sugar": "1.4",\n  "fiber": "0.2",\n  "protein": "0.4",\n  "total fat": "0.9",\n  "saturated fat": "0.4",\n  "unsaturated fat": "0.4",\n  "cholesterol": "0",\n  "glycemic index": "64"\n}\n```\n\nPlease note that the above values are estimates based on typical nutritional information for a Marie biscuit. Actual values may vary between different brands and recipes.',
 'name item': 'Marie biscuit',
 'description of food content': 'A simple, round, thin wheat flour-based biscuit with sugar and vanilla flavor',
 'grams in picture': 6.5,
 'total calories': 27.5,
 'carbohydrates': 4.25,
 'sugar': 0.7,
 'fiber': 0.2,
 'protein': 0.4,
 'total fat': 0.825,
 'saturated fat': 0.25,
 'unsaturated fat': 0.525,
 'cholesterol': 0.0,
 'glycemic in